In [6]:
# URL de páginas de produtos
lista_url = [
    # Disco Rígido HD
    'https://www.kabum.com.br/hardware/disco-rigido-hd?page_number=1&page_size=20&facet_filters=eyJJbnRlcmZhY2UiOlsiU0FUQSJdfQ==&sort=most_searched'
]

In [50]:
from bs4 import BeautifulSoup as bs
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [16]:
r = requests.get("https://www.kabum.com.br/hardware/disco-rigido-hd?page_number=1&page_size=100&facet_filters=eyJJbnRlcmZhY2UiOlsiU0FUQSJdfQ==&sort=most_searched")
soup = bs(r.content)

main = soup.select_one('main.sc-ccc9eb50-13.cWkplc')

articles = []
articles = [article for article in main.select('article.productCard')]

items = []

for article in articles:
    item_url = ('https://www.kabum.com.br/' + article.select_one('a.productLink')['href'] )
    item_name = article.select_one('h3').text
    item_image = article.select_one('a.productLink > img')['src']
    item_openBox = article.find(class_ = 'openboxTagCard')
    if item_openBox:
        item_openBox = True
    else:
        item_openBox = False
    
    # r = requests.get(item_url)
    # item_soup = bs(r.content)
    # item_price = soup.find(class_ = 'finalPrice').text
    # if item_price != None:
    #     item_price =item_price[3:].split('.')        
    #     # Preços de mil reais pra cima
    #     if len(item_price) > 1:
    #         item_price[1] = item_price[1].split(',')
    #         formatted_item_price = float(f"{item_price[0]}{item_price[1][0]}.{item_price[1][1]}")

    #     # Preços abaixo de mil reais
    #     else:
    #         item_price[0] = item_price[0].split(',')
    #         formatted_item_price = float(f"{item_price[0][0]}.{item_price[0][1]}")
    #         item_price = formatted_item_price
    # else:
    #     # O produto não tem preço
    #     item_price = ''
    
    items.append({
        'url': item_url,
        'name': item_name,
        #'price': item_price,
        'image': item_image,
        'openBox': item_openBox
    })

In [39]:
def convert_price_to_float(price_str):
    # Check if the price_str is valid
    if not price_str or '----' in price_str:
        return None
    try:
        # Remove the currency symbol
        price_str = price_str.replace('R$', '').strip()
        # Remove the thousands separator (period)
        price_str = price_str.replace('.', '')
        # Replace comma with period for decimal
        price_str = price_str.replace(',', '.')
        # Convert to float
        price_float = float(price_str)
        return price_float
    except ValueError:
        # Return None if conversion fails
        return None

## Disco Rígido (HD)

In [81]:
URL = 'https://www.kabum.com.br/hardware/disco-rigido-hd?page_number=1&page_size=10&facet_filters=eyJJbnRlcmZhY2UiOlsiU0FUQSJdfQ==&sort=most_searched'

r = requests.get(URL)
soup = bs(r.content)

NUM_PAGES = int(((soup.find('li', class_='next')).find_previous_sibling('li')).text.strip())

# Configurações de Driver
driver = webdriver.Chrome()
driver.get(URL)

# loop de págians

items_2 = []

PAGE = 1
while PAGE <= NUM_PAGES:
    print(f"Processando página {PAGE} de {NUM_PAGES}")
    #main = soup.select_one('main.sc-ccc9eb50-13.cWkplc')

    #articles = []
    #articles = [article for article in main.select('article.productCard')]
    wait = WebDriverWait(driver, 10)
    
    try:
        entendi_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        entendi_button.click()
    except:
        pass
    
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article.productCard')))
    articles = driver.find_elements(By.CSS_SELECTOR, 'article.productCard')

    for article in articles:
        item_url = ('https://www.kabum.com.br/' + article.find_element(By.CSS_SELECTOR, 'a.productLink').get_attribute('href') )
        if item_url:
            item_url = item_url
        else:
            item_url = None 
        
        item_name = article.find_element(By.CSS_SELECTOR, 'h3').text
        item_image = article.find_element(By.CSS_SELECTOR, 'a.productLink > img').get_attribute('src')
        item_openBox = article.find_elements(By.CLASS_NAME, 'openboxTagCard')
        if item_openBox:
            item_openBox = True
        else:
            item_openBox = False
        item_price = article.find_element(By.CLASS_NAME, 'priceCard').text
        item_price = convert_price_to_float(item_price)   
            
        items_2.append({
        'url': item_url,
        'name': item_name,
        'price': item_price,
        'image': item_image,
        'openBox': item_openBox
        })
        
    try:
        next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.next a.nextLink')))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        driver.execute_script("arguments[0].click();", next_button)
        PAGE += 1
    except Exception as e:
        print("Erro ao clicar no botão próximo:", e)
        break
driver.quit()

Processando página 1 de 6
Processando página 2 de 6
Processando página 3 de 6
Processando página 4 de 6
Processando página 5 de 6
Processando página 6 de 6


In [82]:
items_2

[{'url': 'https://www.kabum.com.br/https://www.kabum.com.br/produto/100916/hd-seagate-2tb-barracuda-3-5-sata-st2000dm008',
  'name': "HD Seagate 2TB BarraCuda, 3.5', SATA - ST2000DM008",
  'price': 439.99,
  'image': 'https://images.kabum.com.br/produtos/fotos/100916/100916_1552934430_index_m.jpg',
  'openBox': False},
 {'url': 'https://www.kabum.com.br/https://www.kabum.com.br/produto/95803/hd-seagate-4tb-barracuda-3-5-sata-st4000dm004',
  'name': "HD Seagate 4TB BarraCuda, 3.5', SATA - ST4000DM004",
  'price': 596.99,
  'image': 'https://images.kabum.com.br/produtos/fotos/95803/95803_1522867513_index_m.jpg',
  'openBox': False},
 {'url': 'https://www.kabum.com.br/https://www.kabum.com.br/produto/472212/hd-western-digital-purple-2tb-64mb-sata-3-wd23purz',
  'name': 'HD Western Digital Purple, 2TB, 64mb, Sata 3 - WD23PURZ',
  'price': 429.99,
  'image': 'https://images.kabum.com.br/produtos/fotos/472212/hd-western-digital-purple-2tb-64mb-sata-3-wd23purz_1695294733_m.jpg',
  'openBox': 